In [2]:
# https://www.irscalculators.com/tax-calculator

# self employement taxable amount changes federal taxable income, but not FICA taxes
# Schedule SE line 13
# self employed deduction (Schedule SE line 13)
# 199a deduction

# Taxable income and Federal Taxable Income are different

In [ ]:
# Default Constants
# LATER convert these to json files or something

FILING_STATUS = "single"

# https://www.fidelity.com/learning-center/smart-money/standard-deduction
STANDARD_DEDUCTIONS = {
    "single":                               15000,
    "married_filing_jointly":               30000,
    "married_filing_separately":            15000,
    "head_of_household":                    22500,
    "single_over_65":                       16950,
    "married_filing_jointly_one_over_65":   31550,
    "married_filing_jointly_both_over_65":  33100,
    "married_filing_separately_over_65":    16950,
    "head_of_household_over_65":            24450
}

STANDARD_DEDUCTION = STANDARD_DEDUCTIONS[FILING_STATUS]
SOCIAL_SECURITY_WAGE_BASE = 168600

# Perminant Constants
PROVISIONAL_INCOME_THRESHOLDS = {
        "single":                    (25000, 34000),
        "head_of_household":         (25000, 34000),
        "married_filing_jointly":    (32000, 44000),
        "married_filing_separately": (0, 0)
}

# INCOME INPUTS
INCOME_INPUTS = {
    "wages":            0,
    "self_employment":  0,
    "unearned_income":  0,
    "capital_gains":    0,
    "social_security":  0
}

DEDUCTION_INPUTS = {
    "dependents":           0, 
    "standard_deduction":   STANDARD_DEDUCTION,
    "health & retirement":  0
}

In [9]:
federal_brackets = {
    "single": [
        (11925, 0.10),
        (48475, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250525, 0.32),
        (626350, 0.35),
        (float('inf'), 0.37)
    ],
    "married_joint": [
        (23850, 0.10),
        (96950, 0.12),
        (206700, 0.22),
        (394600, 0.24),
        (501050, 0.32),
        (751600, 0.35),
        (float('inf'), 0.37)
    ],
    "head": [
        (17000, 0.10),
        (64850, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250500, 0.32),
        (626350, 0.35),
        (float('inf'), 0.37)
    ],
    "married_separate": [
        (11925, 0.10),
        (48475, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250525, 0.32),
        (375800, 0.35),
        (float('inf'), 0.37)
    ]
}

In [ ]:
income, deductions = [69000, 0, 0, 0, 0], [STANDARD_DEDUCTION, 0, 0]

class Taxes:
    def __init__(self, federal_brackets, status, income, deductions):
        self.status = status
        self.federal_brackets = federal_brackets
        self.standard_deduction, self.dependents, self.health_and_retirement = deductions
        self.wages, self.self_employment, self.unearned_income, self.capital_gains, self.social_security = income
    
    def taxable_income(self):
        income = self.wages + self.self_employment + self.unearned_income + self.capital_gains + self.social_security
        taxable = income - (self.standard_deduction[self.status] + self.dependents + self.health_and_retirement)
        return taxable

    def federal_tax(self):
        # Import information
        income = self.wages
        federal_brackets = [(0, 0)] + self.federal_brackets[self.status]
        print(federal_brackets)
        bracketed_taxes = []

        # Compute each federal income tax bracket
        for i, bracket in enumerate(federal_brackets[1:]):
            prior_threshold = federal_brackets[i][0]
            threshold = bracket[0]
            marginal_rate = bracket[1]

            marginal_income = max(min(income, threshold) - prior_threshold, 0)
            bracketed_taxes += [marginal_income * marginal_rate]
            

        return bracketed_taxes


irs = Taxes(federal_brackets, 'single', income, deductions)
irs.federal_tax()

[(0, 0), (11925, 0.1), (48475, 0.12), (103350, 0.22), (197300, 0.24), (250525, 0.32), (626350, 0.35), (inf, 0.37)]


[1192.5, 4386.0, 4515.5, 0.0, 0.0, 0.0, 0.0]